In [40]:
import itertools

import pandas as pd
import gffutils
import pybedtools
import re
import numpy as np

v19db_filename = '/projects/ps-yeolab/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

# se_df = pd.read_csv('{}/exons_to_junctions_se.csv'.format(folder), index_col=0)
se_df = pd.read_csv('{}/skipped_exon_consolidated_junctions.csv'.format(folder))

renamer = dict((x, x.replace('junction_exon', 'junction_exons')) for x in se_df)
print renamer
se_df = se_df.rename(columns=renamer)
print se_df.shape
se_df.head()

{'event_id': 'event_id', 'junction_exon23': 'junction_exons23', 'criteria_additional': 'criteria_additional', 'junction_exon13': 'junction_exons13', 'junction_exon12': 'junction_exons12', 'criteria': 'criteria', 'criteria_full': 'criteria_full'}
(28034, 7)


,junction_exons12,junction_exons23,junction_exons13,criteria_full,event_id,criteria,criteria_additional
0,chr10:100189647-100190327:-,chr10:100189400-100189547:-,chr10:100189400-100190327:-,"best,appris_principal",exon:chr10:100190328-100190427:-@exon:chr10:10...,best,appris_principal
1,chr10:100191049-100193696:-,chr10:100190428-100190887:-,chr10:100190428-100193696:-,only one,exon:chr10:100193697-100193848:-@exon:chr10:10...,only one,NaN
2,chr10:100195172-100195391:-,chr10:100193849-100195028:-,chr10:100193849-100195391:-,"best,appris_principal",exon:chr10:100195392-100195529:-@exon:chr10:10...,best,appris_principal
3,chr10:101163632-101165512:-,chr10:101163392-101163480:-,chr10:101163392-101165512:-,only one,exon:chr10:101165513-101165617:-@exon:chr10:10...,only one,NaN
4,chr10:101419346-101419618:+,chr10:101419722-101421202:+,chr10:101419346-101421202:+,only one,exon:chr10:101419263-101419345:+@exon:chr10:10...,only one,NaN


In [41]:
se_exons = se_df.event_id.str.split('@').apply(pd.Series).rename(columns=dict((i, 'exon{}'.format(i+1)) for i in range(3)))
se_df = pd.concat([se_df, se_exons], axis=1)
se_df.head()

,junction_exons12,junction_exons23,junction_exons13,criteria_full,event_id,criteria,criteria_additional,exon1,exon2,exon3
0,chr10:100189647-100190327:-,chr10:100189400-100189547:-,chr10:100189400-100190327:-,"best,appris_principal",exon:chr10:100190328-100190427:-@exon:chr10:10...,best,appris_principal,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100189330-100189399:-
1,chr10:100191049-100193696:-,chr10:100190428-100190887:-,chr10:100190428-100193696:-,only one,exon:chr10:100193697-100193848:-@exon:chr10:10...,only one,NaN,exon:chr10:100193697-100193848:-,exon:chr10:100190888-100191048:-,exon:chr10:100190328-100190427:-
2,chr10:100195172-100195391:-,chr10:100193849-100195028:-,chr10:100193849-100195391:-,"best,appris_principal",exon:chr10:100195392-100195529:-@exon:chr10:10...,best,appris_principal,exon:chr10:100195392-100195529:-,exon:chr10:100195029-100195171:-,exon:chr10:100193697-100193848:-
3,chr10:101163632-101165512:-,chr10:101163392-101163480:-,chr10:101163392-101165512:-,only one,exon:chr10:101165513-101165617:-@exon:chr10:10...,only one,NaN,exon:chr10:101165513-101165617:-,exon:chr10:101163481-101163631:-,exon:chr10:101163226-101163391:-
4,chr10:101419346-101419618:+,chr10:101419722-101421202:+,chr10:101419346-101421202:+,only one,exon:chr10:101419263-101419345:+@exon:chr10:10...,only one,NaN,exon:chr10:101419263-101419345:+,exon:chr10:101419619-101419721:+,exon:chr10:101421203-101421385:+


In [42]:
mxe_df = pd.read_csv('{}/mutually_exclusive_exon_consolidated_events.csv'.format(folder))
renamer = dict((x, x.replace('junction', 'junction_exons')) for x in mxe_df if 'junction' in x)
print renamer
mxe_df = mxe_df.rename(columns=renamer)
print mxe_df.shape
mxe_df.head()

{'junction34': 'junction_exons34', 'junction24': 'junction_exons24', 'junction13': 'junction_exons13', 'junction12': 'junction_exons12'}
(5659, 8)


,junction_exons13,junction_exons34,junction_exons12,junction_exons24,criteria_full,event_id,criteria,criteria_additional
0,chr10:101996727-102003454:-,chr10:101995522-101996606:-,chr10:101997989-102003454:-,chr10:101995522-101997778:-,one event with gencode transcripts,exon:chr10:102003455-102003534:-@exon:chr10:10...,one event with gencode transcripts,NaN
1,chr10:102003535-102013177:-,chr10:101997989-102003454:-,chr10:102010090-102013177:-,chr10:101997989-102010004:-,one event with gencode transcripts,exon:chr10:102013178-102013296:-@exon:chr10:10...,one event with gencode transcripts,NaN
2,chr10:102016234-102020722:-,chr10:102013297-102016018:-,chr10:102020077-102020722:-,chr10:102013297-102019974:-,"random,no good tags",exon:chr10:102020723-102020801:-@exon:chr10:10...,random,no good tags
3,chr10:102016234-102021735:-,chr10:102013297-102016018:-,chr10:102020786-102021735:-,chr10:102013297-102020722:-,only one,exon:chr10:102021736-102021820:-@exon:chr10:10...,only one,NaN
4,chr10:102016234-102021735:-,chr10:102013297-102016018:-,chr10:102020802-102021735:-,chr10:102013297-102020722:-,only one,exon:chr10:102021736-102021820:-@exon:chr10:10...,only one,NaN


In [43]:
mxe_exons = mxe_df.event_id.str.split('@').apply(pd.Series).rename(columns=dict((i, 'exon{}'.format(i+1)) for i in range(4)))
mxe_df = pd.concat([mxe_df, mxe_exons], axis=1)
mxe_df.head()

,junction_exons13,junction_exons34,junction_exons12,junction_exons24,criteria_full,event_id,criteria,criteria_additional,exon1,exon2,exon3,exon4
0,chr10:101996727-102003454:-,chr10:101995522-101996606:-,chr10:101997989-102003454:-,chr10:101995522-101997778:-,one event with gencode transcripts,exon:chr10:102003455-102003534:-@exon:chr10:10...,one event with gencode transcripts,NaN,exon:chr10:102003455-102003534:-,exon:chr10:101997779-101997988:-,exon:chr10:101996607-101996726:-,exon:chr10:101995424-101995521:-
1,chr10:102003535-102013177:-,chr10:101997989-102003454:-,chr10:102010090-102013177:-,chr10:101997989-102010004:-,one event with gencode transcripts,exon:chr10:102013178-102013296:-@exon:chr10:10...,one event with gencode transcripts,NaN,exon:chr10:102013178-102013296:-,exon:chr10:102010005-102010089:-,exon:chr10:102003455-102003534:-,exon:chr10:101997779-101997988:-
2,chr10:102016234-102020722:-,chr10:102013297-102016018:-,chr10:102020077-102020722:-,chr10:102013297-102019974:-,"random,no good tags",exon:chr10:102020723-102020801:-@exon:chr10:10...,random,no good tags,exon:chr10:102020723-102020801:-,exon:chr10:102019975-102020076:-,exon:chr10:102016019-102016233:-,exon:chr10:102013178-102013296:-
3,chr10:102016234-102021735:-,chr10:102013297-102016018:-,chr10:102020786-102021735:-,chr10:102013297-102020722:-,only one,exon:chr10:102021736-102021820:-@exon:chr10:10...,only one,NaN,exon:chr10:102021736-102021820:-,exon:chr10:102020723-102020785:-,exon:chr10:102016019-102016233:-,exon:chr10:102013178-102013296:-
4,chr10:102016234-102021735:-,chr10:102013297-102016018:-,chr10:102020802-102021735:-,chr10:102013297-102020722:-,only one,exon:chr10:102021736-102021820:-@exon:chr10:10...,only one,NaN,exon:chr10:102021736-102021820:-,exon:chr10:102020723-102020801:-,exon:chr10:102016019-102016233:-,exon:chr10:102013178-102013296:-


In [44]:
from poshsplice.region import Region

def get_gene_attributes(df, exon_cols, db):
    df['gencode_id'] = df[exon_cols].apply(
        lambda x: ','.join(set(itertools.chain(*[db[i].attributes['gene_id'] for i in x]))), axis=1)
    df['gene_name'] = df[exon_cols].apply(
        lambda x: ','.join(set(itertools.chain(*[db[i].attributes['gene_name'] for i in x]))), axis=1)
    df['ensembl_id'] = df[exon_cols].apply(
        lambda x: ','.join(set(itertools.chain(*[map(lambda y: y.split('.')[0], db[i].attributes['gene_id']) for i in x]))), 
        axis=1)
    
    for exon_col in exon_cols:
        df['{}_region'.format(exon_col)] = df[exon_col].map(Region)
        df['{}_length'.format(exon_col)] = df['{}_region'.format(exon_col)].map(len)
    df['strand'] = df[exon_cols[0]].str[-1]
    return df

In [45]:
mxe_exon_cols = ['exon{}'.format(i) for i in range(1, 5)]
mxe_df_annotated = get_gene_attributes(mxe_df, mxe_exon_cols, v19db)

In [46]:
se_exon_cols = ['exon{}'.format(i) for i in range(1, 4)]
se_df_annotated = get_gene_attributes(se_df, se_exon_cols, v19db)

In [47]:
mxe_df_annotated.loc[mxe_df_annotated.gene_name == 'PKM']

,junction_exons13,junction_exons34,junction_exons12,junction_exons24,criteria_full,event_id,criteria,criteria_additional,exon1,exon2,...,ensembl_id,exon1_region,exon1_length,exon2_region,exon2_length,exon3_region,exon3_length,exon4_region,exon4_length,strand
1382,chr15:72494962-72499068:-,chr15:72492997-72494794:-,chr15:72495530-72499068:-,chr15:72492997-72495362:-,one event with gencode transcripts,exon:chr15:72499069-72499221:-@exon:chr15:7249...,one event with gencode transcripts,NaN,exon:chr15:72499069-72499221:-,exon:chr15:72495363-72495529:-,...,ENSG00000067225,exon:chr15:72499069-72499221:-,153,exon:chr15:72495363-72495529:-,167,exon:chr15:72494795-72494961:-,167,exon:chr15:72492815-72492996:-,182,-


In [48]:
positive_strand = se_df_annotated.strand == '+'
se_df_annotated['intron_length'] = np.nan


se_df_annotated.loc[positive_strand, 'intron_length'] = \
    se_df_annotated.loc[positive_strand].apply(lambda x: x.exon3_region._start - x.exon1_region._stop - 1, axis=1)
se_df_annotated.loc[~positive_strand, 'intron_length'] = \
    se_df_annotated.loc[~positive_strand].apply(lambda x: x.exon1_region._start - x.exon3_region._stop - 1, axis=1)
se_df_annotated.head()

,junction_exons12,junction_exons23,junction_exons13,criteria_full,event_id,criteria,criteria_additional,exon1,exon2,exon3,...,gene_name,ensembl_id,exon1_region,exon1_length,exon2_region,exon2_length,exon3_region,exon3_length,strand,intron_length
0,chr10:100189647-100190327:-,chr10:100189400-100189547:-,chr10:100189400-100190327:-,"best,appris_principal",exon:chr10:100190328-100190427:-@exon:chr10:10...,best,appris_principal,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100189330-100189399:-,...,HPS1,ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,70,-,928
1,chr10:100191049-100193696:-,chr10:100190428-100190887:-,chr10:100190428-100193696:-,only one,exon:chr10:100193697-100193848:-@exon:chr10:10...,only one,NaN,exon:chr10:100193697-100193848:-,exon:chr10:100190888-100191048:-,exon:chr10:100190328-100190427:-,...,HPS1,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,100,-,3269
2,chr10:100195172-100195391:-,chr10:100193849-100195028:-,chr10:100193849-100195391:-,"best,appris_principal",exon:chr10:100195392-100195529:-@exon:chr10:10...,best,appris_principal,exon:chr10:100195392-100195529:-,exon:chr10:100195029-100195171:-,exon:chr10:100193697-100193848:-,...,HPS1,ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,152,-,1543
3,chr10:101163632-101165512:-,chr10:101163392-101163480:-,chr10:101163392-101165512:-,only one,exon:chr10:101165513-101165617:-@exon:chr10:10...,only one,NaN,exon:chr10:101165513-101165617:-,exon:chr10:101163481-101163631:-,exon:chr10:101163226-101163391:-,...,GOT1,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,166,-,2121
4,chr10:101419346-101419618:+,chr10:101419722-101421202:+,chr10:101419346-101421202:+,only one,exon:chr10:101419263-101419345:+@exon:chr10:10...,only one,NaN,exon:chr10:101419263-101419345:+,exon:chr10:101419619-101419721:+,exon:chr10:101421203-101421385:+,...,ENTPD7,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,183,+,1857


In [49]:
positive_strand = mxe_df_annotated.strand == '+'
mxe_df_annotated['intron_length'] = np.nan


mxe_df_annotated.loc[positive_strand, 'intron_length'] = \
    mxe_df_annotated.loc[positive_strand].apply(lambda x: x.exon4_region._start - x.exon1_region._stop - 1, axis=1)
mxe_df_annotated.loc[~positive_strand, 'intron_length'] = \
    mxe_df_annotated.loc[~positive_strand].apply(lambda x: x.exon1_region._start - x.exon4_region._stop - 1, axis=1)
mxe_df_annotated.head()

,junction_exons13,junction_exons34,junction_exons12,junction_exons24,criteria_full,event_id,criteria,criteria_additional,exon1,exon2,...,exon1_region,exon1_length,exon2_region,exon2_length,exon3_region,exon3_length,exon4_region,exon4_length,strand,intron_length
0,chr10:101996727-102003454:-,chr10:101995522-101996606:-,chr10:101997989-102003454:-,chr10:101995522-101997778:-,one event with gencode transcripts,exon:chr10:102003455-102003534:-@exon:chr10:10...,one event with gencode transcripts,NaN,exon:chr10:102003455-102003534:-,exon:chr10:101997779-101997988:-,...,exon:chr10:102003455-102003534:-,80,exon:chr10:101997779-101997988:-,210,exon:chr10:101996607-101996726:-,120,exon:chr10:101995424-101995521:-,98,-,7933
1,chr10:102003535-102013177:-,chr10:101997989-102003454:-,chr10:102010090-102013177:-,chr10:101997989-102010004:-,one event with gencode transcripts,exon:chr10:102013178-102013296:-@exon:chr10:10...,one event with gencode transcripts,NaN,exon:chr10:102013178-102013296:-,exon:chr10:102010005-102010089:-,...,exon:chr10:102013178-102013296:-,119,exon:chr10:102010005-102010089:-,85,exon:chr10:102003455-102003534:-,80,exon:chr10:101997779-101997988:-,210,-,15189
2,chr10:102016234-102020722:-,chr10:102013297-102016018:-,chr10:102020077-102020722:-,chr10:102013297-102019974:-,"random,no good tags",exon:chr10:102020723-102020801:-@exon:chr10:10...,random,no good tags,exon:chr10:102020723-102020801:-,exon:chr10:102019975-102020076:-,...,exon:chr10:102020723-102020801:-,79,exon:chr10:102019975-102020076:-,102,exon:chr10:102016019-102016233:-,215,exon:chr10:102013178-102013296:-,119,-,7426
3,chr10:102016234-102021735:-,chr10:102013297-102016018:-,chr10:102020786-102021735:-,chr10:102013297-102020722:-,only one,exon:chr10:102021736-102021820:-@exon:chr10:10...,only one,NaN,exon:chr10:102021736-102021820:-,exon:chr10:102020723-102020785:-,...,exon:chr10:102021736-102021820:-,85,exon:chr10:102020723-102020785:-,63,exon:chr10:102016019-102016233:-,215,exon:chr10:102013178-102013296:-,119,-,8439
4,chr10:102016234-102021735:-,chr10:102013297-102016018:-,chr10:102020802-102021735:-,chr10:102013297-102020722:-,only one,exon:chr10:102021736-102021820:-@exon:chr10:10...,only one,NaN,exon:chr10:102021736-102021820:-,exon:chr10:102020723-102020801:-,...,exon:chr10:102021736-102021820:-,85,exon:chr10:102020723-102020801:-,79,exon:chr10:102016019-102016233:-,215,exon:chr10:102013178-102013296:-,119,-,8439


In [50]:

dfs = []
annotated_dfs = se_df_annotated, mxe_df_annotated
for df in annotated_dfs:
    cols = [x  for x in df.columns if 'region' not in x]
    dfs.append(df[cols])

splicing_feature_data = pd.concat(dfs, ignore_index=True)
splicing_feature_data = splicing_feature_data.set_index('event_id')
print splicing_feature_data.shape
splicing_feature_data.tail()

(33693, 21)


,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,...,exon4_length,gencode_id,gene_name,intron_length,junction_exons12,junction_exons13,junction_exons23,junction_exons24,junction_exons34,strand
event_id,,,,,,,,,,,,,,,,,,,,,
exon:chrY:2293238-2293345:-@exon:chrY:2276786-2276854:-@exon:chrY:2260395-2260515:-@exon:chrY:2159543-2159644:-,only one,NaN,only one,ENSGR0000169084,exon:chrY:2293238-2293345:-,108,exon:chrY:2276786-2276854:-,69,exon:chrY:2260395-2260515:-,121,...,102,ENSGR0000169084.8,DHRSX,133593,chrY:2276855-2293237:-,chrY:2260516-2293237:-,NaN,chrY:2159645-2276785:-,chrY:2159645-2260394:-,-
exon:chrY:2709527-2709668:+@exon:chrY:2710206-2710283:+@exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+,only one,NaN,only one,ENSG00000129824,exon:chrY:2709527-2709668:+,142,exon:chrY:2710206-2710283:+,78,exon:chrY:2712118-2712298:+,181,...,98,ENSG00000129824.11,RPS4Y1,4018,chrY:2709669-2710205:+,chrY:2709669-2712117:+,NaN,chrY:2710284-2713686:+,chrY:2712299-2713686:+,+
exon:chrY:2710206-2710283:+@exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+,only one,NaN,only one,ENSG00000129824,exon:chrY:2710206-2710283:+,78,exon:chrY:2712118-2712298:+,181,exon:chrY:2713687-2713784:+,98,...,172,ENSG00000129824.11,RPS4Y1,12357,chrY:2710284-2712117:+,chrY:2710284-2713686:+,NaN,chrY:2712299-2722640:+,chrY:2713785-2722640:+,+
exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2733129-2733286:+,only one,NaN,only one,ENSG00000129824,exon:chrY:2712118-2712298:+,181,exon:chrY:2713687-2713784:+,98,exon:chrY:2722641-2722812:+,172,...,158,ENSG00000129824.11,RPS4Y1,20830,chrY:2712299-2713686:+,chrY:2712299-2722640:+,NaN,chrY:2713785-2733128:+,chrY:2722813-2733128:+,+
exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2733129-2733286:+@exon:chrY:2734834-2735309:+,only one,NaN,only one,ENSG00000129824,exon:chrY:2713687-2713784:+,98,exon:chrY:2722641-2722812:+,172,exon:chrY:2733129-2733286:+,158,...,476,ENSG00000129824.11,RPS4Y1,21049,chrY:2713785-2722640:+,chrY:2713785-2733128:+,NaN,chrY:2722813-2734833:+,chrY:2733287-2734833:+,+


In [51]:
splicing_feature_data['exon2_divisible_by_3'] = splicing_feature_data['exon2_length'].astype(int).mod(3) == 0
splicing_feature_data.head()

,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,...,gencode_id,gene_name,intron_length,junction_exons12,junction_exons13,junction_exons23,junction_exons24,junction_exons34,strand,exon2_divisible_by_3
event_id,,,,,,,,,,,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,70,...,ENSG00000107521.14,HPS1,928,chr10:100189647-100190327:-,chr10:100189400-100190327:-,chr10:100189400-100189547:-,NaN,NaN,-,True
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,only one,NaN,only one,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,100,...,ENSG00000107521.14,HPS1,3269,chr10:100191049-100193696:-,chr10:100190428-100193696:-,chr10:100190428-100190887:-,NaN,NaN,-,False
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,152,...,ENSG00000107521.14,HPS1,1543,chr10:100195172-100195391:-,chr10:100193849-100195391:-,chr10:100193849-100195028:-,NaN,NaN,-,False
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,only one,NaN,only one,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,166,...,ENSG00000120053.9,GOT1,2121,chr10:101163632-101165512:-,chr10:101163392-101165512:-,chr10:101163392-101163480:-,NaN,NaN,-,False
exon:chr10:101419263-101419345:+@exon:chr10:101419619-101419721:+@exon:chr10:101421203-101421385:+,only one,NaN,only one,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,183,...,ENSG00000198018.6,ENTPD7,1857,chr10:101419346-101419618:+,chr10:101419346-101421202:+,chr10:101419722-101421202:+,NaN,NaN,+,False


In [52]:
splicing_feature_data['exon2_divisible_by_3'].sum()/float(splicing_feature_data.shape[0])

0.38883447600391774

In [54]:
print splicing_feature_data.ensembl_id.str.contains(',').sum()
print splicing_feature_data.shape

2072
(33693, 22)


3,928/58,842 splicing events have multiple gene ids associated with them. Since we want to intersect this information with existing gene annotations, we need a *single* id. To do this coarsely and simply, we will take the first gene ID for intersecting purposes.

In [57]:
pd.options.display.max_columns = 50

In [58]:
splicing_feature_data['one_ensembl_id'] = splicing_feature_data.ensembl_id.str.split(',').str[0]
splicing_feature_data.head()

,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,exon4,exon4_length,gencode_id,gene_name,intron_length,junction_exons12,junction_exons13,junction_exons23,junction_exons24,junction_exons34,strand,exon2_divisible_by_3,one_ensembl_id
event_id,,,,,,,,,,,,,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,70,NaN,NaN,ENSG00000107521.14,HPS1,928,chr10:100189647-100190327:-,chr10:100189400-100190327:-,chr10:100189400-100189547:-,NaN,NaN,-,True,ENSG00000107521
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,only one,NaN,only one,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,100,NaN,NaN,ENSG00000107521.14,HPS1,3269,chr10:100191049-100193696:-,chr10:100190428-100193696:-,chr10:100190428-100190887:-,NaN,NaN,-,False,ENSG00000107521
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,152,NaN,NaN,ENSG00000107521.14,HPS1,1543,chr10:100195172-100195391:-,chr10:100193849-100195391:-,chr10:100193849-100195028:-,NaN,NaN,-,False,ENSG00000107521
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,only one,NaN,only one,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,166,NaN,NaN,ENSG00000120053.9,GOT1,2121,chr10:101163632-101165512:-,chr10:101163392-101165512:-,chr10:101163392-101163480:-,NaN,NaN,-,False,ENSG00000120053
exon:chr10:101419263-101419345:+@exon:chr10:101419619-101419721:+@exon:chr10:101421203-101421385:+,only one,NaN,only one,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,183,NaN,NaN,ENSG00000198018.6,ENTPD7,1857,chr10:101419346-101419618:+,chr10:101419346-101421202:+,chr10:101419722-101421202:+,NaN,NaN,+,False,ENSG00000198018


## Add whether this is an SE or MXE event

In [59]:
splicing_feature_data['splice_type'] = splicing_feature_data.index.map(lambda x: 'MXE' if len(x.split('@')) == 4 else 'SE')
splicing_feature_data.head()

,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,exon4,exon4_length,gencode_id,gene_name,intron_length,junction_exons12,junction_exons13,junction_exons23,junction_exons24,junction_exons34,strand,exon2_divisible_by_3,one_ensembl_id,splice_type
event_id,,,,,,,,,,,,,,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,70,NaN,NaN,ENSG00000107521.14,HPS1,928,chr10:100189647-100190327:-,chr10:100189400-100190327:-,chr10:100189400-100189547:-,NaN,NaN,-,True,ENSG00000107521,SE
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,only one,NaN,only one,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,100,NaN,NaN,ENSG00000107521.14,HPS1,3269,chr10:100191049-100193696:-,chr10:100190428-100193696:-,chr10:100190428-100190887:-,NaN,NaN,-,False,ENSG00000107521,SE
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,152,NaN,NaN,ENSG00000107521.14,HPS1,1543,chr10:100195172-100195391:-,chr10:100193849-100195391:-,chr10:100193849-100195028:-,NaN,NaN,-,False,ENSG00000107521,SE
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,only one,NaN,only one,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,166,NaN,NaN,ENSG00000120053.9,GOT1,2121,chr10:101163632-101165512:-,chr10:101163392-101165512:-,chr10:101163392-101163480:-,NaN,NaN,-,False,ENSG00000120053,SE
exon:chr10:101419263-101419345:+@exon:chr10:101419619-101419721:+@exon:chr10:101421203-101421385:+,only one,NaN,only one,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,183,NaN,NaN,ENSG00000198018.6,ENTPD7,1857,chr10:101419346-101419618:+,chr10:101419346-101421202:+,chr10:101419722-101421202:+,NaN,NaN,+,False,ENSG00000198018,SE


In [60]:
splicing_feature_data.tail()

,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,exon4,exon4_length,gencode_id,gene_name,intron_length,junction_exons12,junction_exons13,junction_exons23,junction_exons24,junction_exons34,strand,exon2_divisible_by_3,one_ensembl_id,splice_type
event_id,,,,,,,,,,,,,,,,,,,,,,,,
exon:chrY:2293238-2293345:-@exon:chrY:2276786-2276854:-@exon:chrY:2260395-2260515:-@exon:chrY:2159543-2159644:-,only one,NaN,only one,ENSGR0000169084,exon:chrY:2293238-2293345:-,108,exon:chrY:2276786-2276854:-,69,exon:chrY:2260395-2260515:-,121,exon:chrY:2159543-2159644:-,102,ENSGR0000169084.8,DHRSX,133593,chrY:2276855-2293237:-,chrY:2260516-2293237:-,NaN,chrY:2159645-2276785:-,chrY:2159645-2260394:-,-,True,ENSGR0000169084,MXE
exon:chrY:2709527-2709668:+@exon:chrY:2710206-2710283:+@exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+,only one,NaN,only one,ENSG00000129824,exon:chrY:2709527-2709668:+,142,exon:chrY:2710206-2710283:+,78,exon:chrY:2712118-2712298:+,181,exon:chrY:2713687-2713784:+,98,ENSG00000129824.11,RPS4Y1,4018,chrY:2709669-2710205:+,chrY:2709669-2712117:+,NaN,chrY:2710284-2713686:+,chrY:2712299-2713686:+,+,True,ENSG00000129824,MXE
exon:chrY:2710206-2710283:+@exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+,only one,NaN,only one,ENSG00000129824,exon:chrY:2710206-2710283:+,78,exon:chrY:2712118-2712298:+,181,exon:chrY:2713687-2713784:+,98,exon:chrY:2722641-2722812:+,172,ENSG00000129824.11,RPS4Y1,12357,chrY:2710284-2712117:+,chrY:2710284-2713686:+,NaN,chrY:2712299-2722640:+,chrY:2713785-2722640:+,+,False,ENSG00000129824,MXE
exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2733129-2733286:+,only one,NaN,only one,ENSG00000129824,exon:chrY:2712118-2712298:+,181,exon:chrY:2713687-2713784:+,98,exon:chrY:2722641-2722812:+,172,exon:chrY:2733129-2733286:+,158,ENSG00000129824.11,RPS4Y1,20830,chrY:2712299-2713686:+,chrY:2712299-2722640:+,NaN,chrY:2713785-2733128:+,chrY:2722813-2733128:+,+,False,ENSG00000129824,MXE
exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2733129-2733286:+@exon:chrY:2734834-2735309:+,only one,NaN,only one,ENSG00000129824,exon:chrY:2713687-2713784:+,98,exon:chrY:2722641-2722812:+,172,exon:chrY:2733129-2733286:+,158,exon:chrY:2734834-2735309:+,476,ENSG00000129824.11,RPS4Y1,21049,chrY:2713785-2722640:+,chrY:2713785-2733128:+,NaN,chrY:2722813-2734833:+,chrY:2733287-2734833:+,+,False,ENSG00000129824,MXE


### Write to a safe place

In [ ]:
splicing_feature_data.to_csv('{}/splicing_feature_data.csv'.format(folder))